### 자영업자를 위한 소비 분석 카테고리 데이터 분석

1. **대분류별 제품 다양성 분석**:
   - 특정 대분류에서 다양한 제품이 존재하는지 확인할 수 있습니다. 예를 들어, 특정 카테고리에서 다양한 상품을 취급할수록 더 많은 판매 기회가 생길 수 있습니다.
   - 제품 다양성이 적은 카테고리에 대해서는 신제품을 추가하거나, 해당 카테고리에 집중된 마케팅을 고려해볼 수 있습니다.

2. **대분류 및 중분류별 총 판매량 분석**:
   - 판매량이 많은 대분류와 중분류를 파악함으로써 어떤 제품군이 주력 상품인지 알 수 있습니다.
   - 판매량이 낮은 카테고리는 개선이 필요하며, 추가적인 프로모션이나 패키지 상품으로 고객 유치를 고려할 수 있습니다.

3. **대분류-중분류 간 판매량 관계 분석**:
   - 특정 대분류와 중분류 조합에서 판매량이 높은지 확인할 수 있습니다. 이를 통해 자영업자는 인기 있는 조합의 상품군을 중점적으로 프로모션할 수 있습니다.

4. **판매량 상위 제품 분석**:
   - 가장 많이 팔린 상위 제품을 파악함으로써 어떤 상품이 고객의 선호도가 높은지 알 수 있습니다. 이 상품들에 대해 추가적인 재고를 확보하거나, 관련 상품을 패키지로 묶어 판매를 촉진할 수 있습니다.

5. **농산물과 가공상품 간 판매량 비교**:
   - 농산물과 가공상품 간의 판매량 차이를 비교하여, 주력으로 삼아야 할 제품군을 결정할 수 있습니다. 예를 들어, 농산물이 더 많이 팔린다면 신선한 농산물을 주력 상품으로 삼고, 이를 홍보할 수 있습니다.

이러한 분석을 통해 자영업자는 판매 성과를 최적화할 수 있는 구체적인 전략을 도출할 수 있으며, 이는 매출 증대와 고객 만족도 향상에 기여할 수 있습니다.

## 소비 분석 카테고리 데이터
* https://kadx.co.kr/opmk/frn/pmumkproductDetail/PMU_5fe8f2b8-47aa-43a6-8daf-ab9c2e9b84c2/5#!
* [장보자닷컴](http://jangboja.com/)

In [ ]:
# !pip install -Uq koreanize-matplotlib

In [ ]:
import pandas as pd
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from glob import glob

In [ ]:
# 데이터 컬럼 정보를 불러옵니다.
df_col = pd.read_csv("data/kadx-market-consumer-category_detail.csv")
df_col.shape

In [ ]:
df_col

In [ ]:
file_list = glob("data/market-category/*.csv")
file_list = sorted(file_list)
file_list

In [ ]:
df_list = []
for file_path in file_list:
    df_temp = pd.read_csv(file_path, encoding='cp949', low_memory=False)
    df_list.append(df_temp)

In [ ]:
raw = pd.concat(df_list)
raw.shape

In [ ]:
# 영문 컬럼명을 한글로 변환하기 위한 매핑 딕셔너리를 생성합니다.
col_dict = df_col[["nameEn","nameKo"]].set_index("nameEn")['nameKo'].to_dict()
col_dict

In [ ]:
# 원본 데이터의 컬럼명을 한글로 변환합니다.
df = raw.rename(columns=col_dict)

In [ ]:
# 변환된 데이터의 첫 몇 줄을 확인합니다.
df.head()

In [ ]:
df.columns

## 기초 기술통계

In [ ]:
df.info()

In [ ]:
df.describe().round(2)

In [ ]:
df.hist(bins=50, figsize=(10, 8));

In [ ]:
df.describe(exclude="number")

### 식자재품목 대중소분류 분석

In [ ]:
# 대분류별 제품 다양성 분석
product_diversity = df.groupby('식자재품목대분류명')['식자재품목명'].nunique().sort_values(ascending=False)
product_diversity.plot(kind='bar', rot=0, figsize=(10, 2))
plt.title('대분류별 제품 다양성')
plt.show()

In [ ]:
# 대분류별 판매량 분석
subcategory_sales = df.groupby('식자재품목대분류명')['총판매수량'].sum().sort_values(ascending=False).head(10)
subcategory_sales.plot(kind='bar', rot=0, figsize=(10, 2))
plt.title('대분류별 총 판매수량');

In [ ]:
# 중분류별 상위 판매량 분석
subcategory_sales = df.groupby('식자재품목중분류명')['총판매수량'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 2))
subcategory_sales.plot(kind='bar', rot=0)
plt.title('중분류별 총 판매수량');

In [ ]:
# 중분류별 상위 판매량 분석
subcategory_sales = df.groupby('식자재품목소분류명')['총판매수량'].sum().sort_values(ascending=True).tail(20)
plt.figure(figsize=(8, 6))
subcategory_sales.plot(kind='barh', rot=0)
plt.title('소분류별 총 판매수량');

### 식자재 품목명 상위 판매 수량

In [ ]:
# 판매량 상위 제품 분석
top_products = df.groupby(['식자재품목명'])['총판매수량'].sum().nlargest(30).dropna()
top_products

### 대중소 분류 시각화

In [ ]:
# 대분류-중분류 빈도 분석
category_subcategory = df.groupby(
    ['식자재품목대분류명', '식자재품목중분류명', '식자재품목소분류명'])['총판매수량'].sum().reset_index()
category_subcategory

In [ ]:
import plotly.express as px

# Data preparation
df_reset = category_subcategory.reset_index()  # Reset index to prepare for Plotly

# Sunburst plot
fig_sunburst = px.sunburst(
    df_reset,
    path=['식자재품목대분류명', '식자재품목중분류명', '식자재품목소분류명'],
    values='총판매수량',
    title="Sunburst Plot of Sales by Category Hierarchy"
)
fig_sunburst.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig_sunburst.show()

# Treemap plot
fig_treemap = px.treemap(
    df_reset,
    path=['식자재품목대분류명', '식자재품목중분류명', '식자재품목소분류명'],
    values='총판매수량',
    title="Treemap of Sales by Category Hierarchy"
)
fig_treemap.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig_treemap.show()


## 확정일자

In [ ]:
df['확정일자']

In [ ]:
df['확정일자'] = pd.to_datetime(df['확정일자'], format='%Y%m%d')
df['확정일자'].describe()

In [ ]:
df['월'] = df['확정일자'].dt.month
df['요일'] = df['확정일자'].dt.dayofweek

In [ ]:
# 일별 판매량 합계 계산
daily_sales = df.groupby('확정일자')['총판매수량'].sum()
daily_sales.plot(figsize=(10, 2));

### 확정일자별 주문

In [ ]:
mdow = pd.crosstab(df['월'], df['요일'])
mdow.columns = list('월화수목금토일')
mdow.style.background_gradient(axis=None).format('{:,}')

In [ ]:
mdow = pd.crosstab(df['월'], df['요일'], aggfunc='sum', values=df['총판매수량'])
mdow.columns = list('월화수목금토일')
mdow.style.background_gradient(axis=None).format('{:,}')

In [ ]:
mdow = pd.crosstab(df['식자재품목대분류명'], df['요일'])
mdow.columns = list('월화수목금토일')
mdow.style.background_gradient(axis=None, cmap='Greens').format('{:,}')

In [ ]:
mdow = pd.crosstab(df['식자재품목대분류명'], df['요일'], aggfunc='sum', values=df['총판매수량'])
mdow.columns = list('월화수목금토일')
mdow.style.background_gradient(axis=None, cmap='Greens').format('{:,}')

In [ ]:
mdow.T.plot(subplots=True);

In [ ]:
mdow = pd.crosstab(df['식자재품목중분류명'], df['요일'])
mdow.columns = list('월화수목금토일')
mdow = mdow.sort_values('토', ascending=False)
mdow.style.background_gradient(axis=None, cmap='Greens').format('{:,}')

In [ ]:
mdow = pd.crosstab(df['식자재품목중분류명'], df['요일'], aggfunc='sum', values=df['총판매수량'])
mdow.columns = list('월화수목금토일')
mdow = mdow.sort_values('토', ascending=False)
mdow.style.background_gradient(axis=None, cmap='Greens').format('{:,}')

## 특정 두 가지 상품 비교

In [ ]:
df.columns

In [ ]:
df_sub = df[df['식자재품목소분류명'].isin(['삼겹살', '목심'])]
df_sub.head(3)

In [ ]:
daily_sum = df_sub.groupby(['확정일자','식자재품목소분류명'])['총판매수량'].sum().unstack()
daily_sum.plot(figsize=(10, 3))

## 두 제품군 간 판매량 차이의 통계적 분석 및 시각화
1. 기술 통계:
   * 두 그룹의 중앙값(median)과 사분위수 범위(IQR)를 비교할 수 있습니다.
   * 각 그룹의 평균 순위(mean rank)를 계산하여 데이터의 전반적인 분포 차이를 확인할 수 있습니다.

2. Mann-Whitney U 검정 결과:
   * Mann-Whitney U 통계량과 p-값을 계산할 수 있습니다.
   * 사용자가 지정한 유의수준(기본값 0.05)과 비교하여 결과를 해석할 수 있습니다.

3. 결과 해석:
   * p-값이 유의수준보다 작으면 귀무가설을 기각하고, 두 그룹의 분포가 통계적으로 유의미하게 다르다고 결론 내릴 수 있습니다.
   * p-값이 유의수준보다 크면 귀무가설을 기각할 수 없으며, 두 그룹의 분포에 통계적으로 유의미한 차이가 없다고 결론 내릴 수 있습니다.

4. 시각화:
   * 박스플롯을 통해 두 그룹의 데이터 분포를 시각적으로 비교할 수 있습니다.
   * 이를 통해 각 그룹의 중앙값, 사분위수 범위, 이상치 등을 한눈에 파악할 수 있습니다.

5. 효과 크기:
   * Cliff's delta를 계산하여 두 그룹 간 차이의 실질적 크기를 평가할 수 있습니다.
   * 일반적인 해석 기준(|d| < 0.147 "무시할 만한", |d| < 0.33 "작은", |d| < 0.474 "중간", 그 외 "큰" 효과)을 사용하여 효과의 크기를 판단할 수 있습니다.

6. 추가 분석:
   * 그룹별 판매량에 대한 기술 통계(중앙값, IQR, 데이터 수 등)를 계산하여 전체적인 분포를 파악할 수 있습니다.

7. 전체 데이터 시각화:
   * 모든 그룹에 대한 데이터 분포를 박스플롯으로 시각화하여 전체적인 비교가 가능합니다.

이 코드는 두 그룹 간의 분포 차이를 분석하는 일반적인 프레임워크를 제공합니다. Mann-Whitney U 검정은 데이터가 정규 분포를 따르지 않거나 순서형 데이터일 때 특히 유용합니다. 데이터셋이 바뀌더라도 그룹을 나타내는 열과 비교하고자 하는 수치 데이터 열만 적절히 지정하면, 동일한 분석을 수행할 수 있습니다.

이러한 분석을 통해 데이터의 전반적인 특성을 이해하고, 그룹 간 차이의 통계적 유의성과 실질적 크기를 평가할 수 있습니다. Mann-Whitney U 검정은 중앙 경향값의 차이를 검정하므로, 두 그룹의 전체적인 분포 차이를 파악하는 데 도움이 됩니다. 이는 데이터에 기반한 의사결정을 내리는 데 유용하며, 특히 데이터가 정규 분포를 따르지 않거나 이상치가 많은 경우에 더욱 적합할 수 있습니다.

In [ ]:
sns.boxplot(x='식자재품목소분류명', y='총판매수량', data=df_sub)

In [ ]:
sns.histplot(data=df_sub, x='총판매수량', hue='식자재품목소분류명')

In [ ]:
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

# 그룹 분리
pork_belly = df[df['식자재품목소분류명'] == '삼겹살']['총판매수량']
pork_neck = df[df['식자재품목소분류명'] == '목심']['총판매수량']

# 기술 통계량 계산
belly_mean = pork_belly.mean()
neck_mean = pork_neck.mean()
belly_std = pork_belly.std()
neck_std = pork_neck.std()

print(f"삼겹살 평균 판매량: {belly_mean:.2f}, 표준편차: {belly_std:.2f}")
print(f"목심 평균 판매량: {neck_mean:.2f}, 표준편차: {neck_std:.2f}")

In [ ]:
# 정규성 검정
stat_belly, p_belly = shapiro(pork_belly)
stat_neck, p_neck = shapiro(pork_neck)
print(f"삼겹살 판매량 정규성 검정 p-값: {p_belly:.4f}")
print(f"목심 판매량 정규성 검정 p-값: {p_neck:.4f}")

# 등분산 검정
stat_levene, p_levene = levene(pork_belly, pork_neck)
print(f"등분산 검정 p-값: {p_levene:.4f}")

In [ ]:
# 정규성에 따라 적절한 검정 방법 선택
if p_belly > 0.05 and p_neck > 0.05:
    # t-검정 수행
    equal_var = True if p_levene > 0.05 else False
    stat_test, p_value = ttest_ind(pork_belly, pork_neck, equal_var=equal_var)
    test_name = 't-검정'
else:
    # Mann-Whitney U 검정 수행
    stat_test, p_value = mannwhitneyu(pork_belly, pork_neck)
    test_name = 'Mann-Whitney U 검정'

print(f"\n{test_name} 결과: p-값 = {p_value:.4f}")

# 결과 해석
alpha = 0.05
if p_value < alpha:
    print("판매수량에 유의미한 차이가 있습니다.")
else:
    print("판매수량에 유의미한 차이가 없습니다.")

# 회귀분석 준비
pork_belly_df = pd.DataFrame({'판매수량': pork_belly, '품목': '삼겹살'})
pork_neck_df = pd.DataFrame({'판매수량': pork_neck, '품목': '목심'})

data = pd.concat([pork_belly_df, pork_neck_df], ignore_index=True)

# 회귀분석 수행
model = smf.ols('판매수량 ~ C(품목)', data=data).fit()
print(model.summary())

# 분산분석표 생성
anova_results = anova_lm(model)
print(anova_results)

## 추가로 분석해 볼 수 있는 것
* 전체 카테고리 비교: 전체적인 제품 카테고리 성과를 비교할 수 있습니다.
데이터 불균형 확인: 각 대분류별 데이터 수(count)를 통해 특정 카테고리에 데이터가 편중되어 있는지 확인할 수 있습니다. 이는 분석 결과의 신뢰성을 평가하는 데 도움이 될 수 있습니다.
* 변동성 비교: 각 대분류의 표준편차를 비교함으로써, 어떤 카테고리의 판매량이 더 안정적인지 또는 변동이 큰지 파악할 수 있습니다.
* 이상치 식별: 박스플롯을 통해 각 대분류에서 이상치(outlier)를 시각적으로 확인할 수 있습니다. 이는 특별히 높거나 낮은 판매량을 기록한 제품을 식별하는 데 도움이 될 수 있습니다.
* 전략적 포커스: 평균 판매량이 높은 대분류에 대해서는 현재의 성공 요인을 분석하고, 낮은 대분류에 대해서는 개선 전략을 수립할 수 있습니다.

이러한 분석을 통해 더 넓은 시각에서 제품 카테고리 전반에 대한 이해를 높이고, 데이터에 기반한 전략적 의사결정을 내릴 수 있습니다. 각 대분류의 특성과 성과를 비교함으로써, 제품 라인업 조정, 재고 관리 최적화, 마케팅 전략 수립 등 다양한 비즈니스 영역에서 인사이트를 활용할 수 있습니다.